In [6]:
import heapq
import networkx as nx
from math import ceil

In [7]:
main_graph = nx.read_graphml("grafo_ponte_tocha.graphml")
main_crossing_times = {'A': 1, 'B': 2, 'C': 5, 'D': 10}

In [8]:
def parse_state(node_str: str):
    """
    Convert a node string from GraphML into a state tuple.
    Example: "inicio=ABCD|tocha=inicio" -> (frozenset({'A','B','C','D'}), 'inicio')
    """
    left, right = node_str.split("|")
    people_str = left.split("=")[1].strip()
    torch_pos = right.split("=")[1].strip()

    if people_str == "VAZIO":
        people = frozenset()
    else:
        people = frozenset(list(people_str))

    return (people, torch_pos)

In [9]:
def a_star(graph: nx.DiGraph, start: str, goal: str, heuristic):
    """
    A* algorithm for NetworkX graphs.
    :param graph: NetworkX graph (nx.Graph ou nx.DiGraph)
    :param start: Initial node
    :param goal: Goal node
    :param heuristic: Heuristic function h(node)
    :return: (path, total_cost) or (None, inf) if there is no solution
    """

    # frontier = priority queue of (f, g, path)
    frontier = []
    heapq.heappush(frontier, (heuristic(parse_state(start)), 0, [start]))

    # known accumulated costs
    g_cost = {start: 0}

    while frontier:
        f, g, path = heapq.heappop(frontier)
        current = path[-1]

        # check goal
        if current == goal:
            return path, g

        # expand neighbors
        for neighbor in graph.neighbors(current):
            # edge cost
            edge_cost = graph[current][neighbor].get("weight")
            new_g = g + edge_cost

            if neighbor not in g_cost or new_g < g_cost[neighbor]:
                g_cost[neighbor] = new_g
                new_f = new_g + heuristic(parse_state(neighbor))
                new_path = path + [neighbor]
                heapq.heappush(frontier, (new_f, new_g, new_path))

    return None, float("inf")

def heuristic_func(state):
    """
    Admissible heuristic for the Bridge and Torch problem.
    :param state: (frozenset of people on the start side, torch position)
    :return: heuristic estimate of the remaining cost
    """
    people_start, _ = state
    m = len(people_start)
    if m == 0:
        return 0

    slowest = max(main_crossing_times[p] for p in people_start)
    fastest = min(main_crossing_times.values())

    needed_forward = ceil(m / 2) # minimum number of forward crossings
    min_returns = max(0, needed_forward - 1) # minimum number of returns
    estimated_returns = min_returns * fastest

    return max(slowest, estimated_returns)

In [27]:
start_str = "inicio=ABCD|tocha=inicio"
goal_str = "inicio=VAZIO|tocha=final"

if start_str not in main_graph or goal_str not in main_graph:
    raise ValueError("Start or Goal not in graph.")

final_path, final_cost = a_star(graph=main_graph, start=start_str, goal=goal_str, heuristic=heuristic_func)
print(f"Caminho: {final_path}")
print(f"Custo total: {final_cost}")

Caminho: ['inicio=ABCD|tocha=inicio', 'inicio=CD|tocha=final', 'inicio=ACD|tocha=inicio', 'inicio=A|tocha=final', 'inicio=AB|tocha=inicio', 'inicio=VAZIO|tocha=final']
Custo total: 17
